In [1]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
%matplotlib inline

from scipy.stats import norm
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
from sklearn.metrics import average_precision_score

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
torch.manual_seed(1);
from torch.utils import data

import warnings
warnings.simplefilter("ignore", UserWarning)

## Read data

In [3]:
import sys
from dataloader import get_video_loader, train_transform, test_transform, unnormalize
import argparse

%load_ext autoreload
%autoreload 1
%aimport dataloader


In [4]:
parser = argparse.ArgumentParser(description='',
                                 formatter_class=argparse.ArgumentDefaultsHelpFormatter)

parser.add_argument('--batch_size', default=16, type=int, metavar='N',
                    help='batchsize')
parser.add_argument('-j', '--workers', default=0, type=int, metavar='N',
                    help='number of data loading workers')
parser.add_argument('-c', '--checkpoint',
                    default="checkpoint_bincls_newborders", type=str,
                    metavar='PATH',
                    help='path to save checkpoint')
parser.add_argument('--frames_per_clip', type=int, default=16)
parser.add_argument('--step_between_clips', type=int, default=16)
parser.add_argument('--fps', type=int, default=8)
parser.add_argument('--local_rank', type=int, default=-1)
parser.add_argument('--clips_per_video', type=int, default=1)
parser.add_argument('--fails_path', default="train/")
parser.add_argument('--dataset_path', default="train/")

_StoreAction(option_strings=['--dataset_path'], dest='dataset_path', nargs=None, const=None, default='train/', type=None, choices=None, help=None, metavar=None)

In [5]:
#args = parser.parse_args()
#args, unknown = parser.parse_known_args()

## Loss function

In [6]:
device = torch.device('cuda')

In [7]:
def loss_delay_detection_2(p_slice):
    n = p_slice.size(0)
    prod = torch.ones(n).to(device)
    p_slice = p_slice.to(device)
    prod[1:] -= p_slice[:-1]
    cumprod = torch.cumprod(prod, dim=0).to(device)
    loss = torch.arange(1, n + 1).to(device) * p_slice * cumprod
    loss = torch.sum(loss)
    return loss


def loss_delay_detection(p_slice, w):
    n = p_slice.size(0) 
    prod = torch.ones(n).to(device)
    p_slice = p_slice.to(device)
    
    prod[1:] -= p_slice[:-1].to(device)
    cumprod = torch.cumprod(prod, dim=0).to(device)
    # TODO drop either n or w
    loss = (torch.arange(1, n + 1).to(device) * p_slice * cumprod 
            + (w + 1) * torch.prod(prod[1:]) * (1 - p_slice[-1]))
    loss = torch.sum(loss)
    return loss

def loss_false_alarms(p_slice):
    length = len(p_slice)
    loss = 0
    
    start_ind = 0
    end_ind = 0
    
    while end_ind < length:
        start_ind = end_ind - 1
        if start_ind < 0:
            start_ind = 0
        end_ind = np.random.randint(start_ind, length + 2)
        if end_ind == start_ind:
            end_ind = end_ind + 1
        loss += 1 - loss_delay_detection_2(p_slice[start_ind: end_ind + 1])    
    #loss += 1 - loss_delay_detection_2(p_slice[start_ind: length])            
    loss = torch.sum(loss)
    return loss

In [8]:
class CustomLoss(nn.Module):
    
    def __init__(self):
        super(CustomLoss, self).__init__()
        
    def forward(self, outputs, labels):
        loss = torch.zeros(labels.size(0))
        for i, label in enumerate(labels):
            ind = torch.where(label != label[0])[0]
            if ind.size()[0] == 0:
                loss[i] = loss_false_alarms(outputs[i, :])
            else:
                w = 16
                alpha = 0.5
                loss[i] = (alpha * loss_delay_detection(outputs[i, ind[0]:(ind[0] + w)], w) + 
                           (1 - alpha) * loss_false_alarms(outputs[i, :ind[0]]))
        loss = torch.mean(loss)
        return loss


## Model

In [9]:
import torchvision

In [10]:
mobile_net_v2 = torch.hub.load('pytorch/vision:v0.6.0', 'mobilenet_v2', pretrained=True)

class FullyConnected(torch.nn.Module):
    def __init__(self):
        super(FullyConnected, self).__init__()
        self.clas_block1 = nn.Sequential(
            nn.Dropout2d(0.2),
            nn.Linear(1280, 100)
        )
        
    def forward(self, x):
        x = self.clas_block1(x)
        return x.view(-1, 100)

mobile_net_v2.classifier = FullyConnected()

Using cache found in /home/eromanenkova/.cache/torch/hub/pytorch_vision_v0.6.0


In [11]:
class Combine(nn.Module):
    def __init__(self):
        super(Combine, self).__init__()
        self.cnn = mobile_net_v2
        self.rnn = nn.LSTM(
            input_size=100,  # 1280 if conv block mobile net used
            hidden_size=50, 
            num_layers=1,
            batch_first=True, 
            dropout=0.5)
        self.linear_1 = nn.Linear(100,50)
        self.linear_2 = nn.Linear(50,1)
        self.linear = nn.Linear(50,1)        
        self.dropout = nn.Dropout(0.5)
        self.relu = nn.ReLU()  
        self.activation = nn.Sigmoid()        

    def forward(self, x):
        batch_size, C, timesteps, H, W = x.size()
        x = x.permute(0,2,1,3,4)
        c_in = x.reshape(batch_size * timesteps, C, H, W)
        c_in = torchvision.transforms.Resize((224, 224))(c_in)
        c_out = self.cnn(c_in)        
        
        #print(c_out.shape)
        r_in = c_out.view(batch_size, timesteps, -1)
        r_out, hidden = self.rnn(r_in)
        #print(r_out.shape)
        r_out2 = self.linear(r_out.reshape(batch_size * timesteps, -1)).reshape(batch_size, timesteps, -1)
        out = self.activation(r_out2)
        return out
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        
        hidden = (weight.new(1, batch_size, 100).zero_().to(device),
                  weight.new(1, batch_size, 100).zero_().to(device))
        return hidden

In [12]:
model = Combine() 

cnt = 0
for param in model.parameters():
    cnt+=1
    if(cnt==157): break   
    param.requires_grad = False

In [13]:
device = 'cuda:1'
model = Combine()
model.to(device)

Combine(
  (cnn): MobileNetV2(
    (features): Sequential(
      (0): ConvBNActivation(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
      )
      (1): InvertedResidual(
        (conv): Sequential(
          (0): ConvBNActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU6(inplace=True)
          )
          (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (2): InvertedResidual(
        (conv): Sequential(
          (0): ConvBNActivation(
            (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1),

In [14]:
#loss_function = CustomLoss()
loss_function  = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [22]:
import pytorch_lightning as pl
import torch
import torch.nn as nn

class CPD_model(pl.LightningModule):
    def __init__(
        self,
        lr: float = None,
        batch_size: int = 16,
    ):
        super(CPD_model, self).__init__()

        self.model = Combine()                
        self.lr = lr
        self.batch_size = batch_size
    def forward(self, inputs):
        return self.model(inputs)

    def training_step(self, batch, batch_idx):
        videos, labels, _ = batch
        pred = self.forward(videos.float())

        train_loss = nn.BCELoss()(pred.squeeze(), labels.float().squeeze())
        train_accuracy = ((pred.squeeze() > 0.5).long() == labels.squeeze()).float().mean()

        self.log('train_loss', train_loss, prog_bar=True)
        self.log('train_acc', train_accuracy, prog_bar=True)    
        
        return train_loss

    def validation_step(self, batch, batch_idx):
        videos, labels, _ = batch
        pred = self.forward(videos.float())
        
        val_loss = nn.BCELoss()(pred.squeeze(), labels.float().squeeze())      
        val_accuracy = ((pred.squeeze() > 0.5).long() == labels.squeeze()).float().mean()
                
        self.log('val_loss', val_loss, prog_bar=True)    
        self.log('val_acc', val_accuracy, prog_bar=True)    
        
        return {'val_loss': val_loss, 'val_acc': val_accuracy}
    

    def configure_optimizers(self):
        opt = torch.optim.Adam(self.model.parameters(), lr=0.0001) 
        stp = torch.optim.lr_scheduler.StepLR(opt, 5, gamma=0.9)

        return [opt], [stp]

    def train_dataloader(self):
        
        train_args = ['--batch_size', str(self.batch_size), '--workers', '2', '--checkpoint', 'checkpoint_bincls_newborders', 
                      '--frames_per_clip', '16', '--step_between_clips', '16', '--fps', '8',
                      '--local_rank', '-1', '--clips_per_video', '1', '--fails_path', 'train/']
        train_args = parser.parse_args(train_args)
        return get_video_loader(**vars(train_args), val=False, transform=train_transform)
    
    def val_dataloader(self):
        
        test_args = ['--batch_size', str(self.batch_size), '--workers', '2', '--checkpoint', 'checkpoint_bincls_newborders', 
                     '--frames_per_clip', '16', '--step_between_clips', '16', '--fps', '8',
                     '--local_rank', '-1', '--clips_per_video', '1', '--fails_path', 'val/']
        test_args = parser.parse_args(test_args)
        return get_video_loader(**vars(test_args), val=True, transform=test_transform)    

In [16]:
experiment_name = 'oops_small'
logger = pl.loggers.TensorBoardLogger(save_dir='logs/', name=experiment_name)

trainer = pl.Trainer(
    max_epochs=100, 
    gpus='1', 
    benchmark=True, 
    check_val_every_n_epoch=1, 
    logger=logger)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


In [17]:
%load_ext tensorboard 
%tensorboard --logdir logs --port 7777

Reusing TensorBoard on port 7777 (pid 7658), started 2 days, 5:45:22 ago. (Use '!kill 7658' to kill it.)

In [18]:
model = CPD_model()

In [19]:
trainer.fit(model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
/home/eromanenkova/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/eromanenkova/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/eromanenkova/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/eromanenkova/a

  1%|          | 12/1233 [00:00<00:10, 119.25it/s]

Loaded dataset from train/val_videoclips.pth


100%|██████████| 1233/1233 [00:08<00:00, 153.24it/s]

Good clips: 2371
Only ones: 1629
Without labels: 309



  0%|          | 0/6169 [00:00<?, ?it/s]

Loaded dataset from train/train_videoclips.pth


100%|██████████| 6169/6169 [00:41<00:00, 149.10it/s]


Good clips: 10913
Only ones: 7432
Without labels: 1757
Dataset contains 10913 items


In [ ]:
experiment_name = 'oops_small'
logger = pl.loggers.TensorBoardLogger(save_dir='logs/', name=experiment_name + '2')

checkpoint_callback = pl.callbacks.ModelCheckpoint(
    monitor='val_loss',
    dirpath=f'logs/{experiment_name}',
    filename='{epoch:02d}-{val_loss:.3f}',
    mode='min')

trainer = pl.Trainer(
    max_epochs=100, 
    gpus='1', 
    benchmark=True, 
    check_val_every_n_epoch=1, 
    logger=logger,
    callbacks=[checkpoint_callback])

In [24]:
model.load_state_dict('bce_loss_100_epochs.pth')

AttributeError: 'str' object has no attribute 'copy'

In [ ]:
trainer.fit(model)